### Make the necessary imports (libraries)

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras as k

from keras import layers, models, optimizers
from sklearn.metrics import classification_report, confusion_matrix
from keras.utils import to_categorical
from keras import regularizers


import os
import time
import tarfile
import pickle
import gzip
import requests



In [ ]:
def download_cifar10():
  working_path = os.getcwd()
  cifar_url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
  file_name_gz = "cifar-10-python.tar.gz"
  file_name_tar = "cifar-10-python.tar"

  file_path_gz = os.path.join(working_path, file_name_gz)
  file_path_tar = os.path.join(working_path, file_name_tar)

  if os.path.exists(file_path_tar):
      print("The file cifar-10-python.tar already exists")
  else:
      print("Downloading CIFAR-10 data...")
      response = requests.get(cifar_url, stream=True)
      response.raise_for_status()
      with open(file_path_gz, 'wb') as f:
          for chunk in response.iter_content(chunk_size=8192):
              if chunk:
                  f.write(chunk)

      with gzip.open(file_path_gz, 'rb') as f_in:
          with open(file_path_tar, 'wb') as f_out:
              for chunk in f_in:
                  f_out.write(chunk)
      print
  return file_path_tar

download_cifar10()

### Load the data from a tar file (using pickle)

In [ ]:
def load_pickle_from_tar(tar_path, pickle_path):
    with tarfile.open(tar_path, 'r') as tar:
        member = tar.getmember(pickle_path)
        f = tar.extractfile(member)
        data_dict = pickle.load(f, encoding='bytes')
    return data_dict

# Set your tar_path to the file in the same working folder
tar_path = "cifar-10-python.tar"  # File in the same working directory

# Load training data
train_data = []
train_labels = []

print("Loading training data...")
# The path structure below is based on the standard CIFAR-10 distribution
for i in range(1, 6):
    batch_path = f'cifar-10-batches-py/data_batch_{i}'
    print(f"Loading batch {i}...")
    batch_dict = load_pickle_from_tar(tar_path, batch_path)
    train_data.append(batch_dict[b'data'])
    train_labels.extend(batch_dict[b'labels'])

train_data = np.vstack(train_data)

### Load the training/test data set and label names

In [ ]:
print("Loading test data...")
test_batch_path = 'cifar-10-batches-py/test_batch'
test_dict = load_pickle_from_tar(tar_path, test_batch_path)
test_data = test_dict[b'data']
test_labels = test_dict[b'labels']

# Load label names
print("Loading label names...")
meta_path = 'cifar-10-batches-py/batches.meta'
meta_dict = load_pickle_from_tar(tar_path, meta_path)
label_names = [label.decode('utf-8') for label in meta_dict[b'label_names']]

# Print basic info about the data
print(f"Training data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")
print(f"Label names: {label_names}")


### Reshape the data to match the expected input format for CNN

In [ ]:

# Each image is 32x32 pixels with 3 color channels (RGB)
X_train = train_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)  # Reshape to (N, 32, 32, 3)
X_test = test_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)  # Reshape to (N, 32, 32, 3)
y_train = np.array(train_labels)
y_test = np.array(test_labels)

# Print updated shapes
print("Training data shape after reshaping:", X_train.shape)
print("Test data shape after reshaping:", X_test.shape)


### Conduct preprocessing of the data:
- Normalization (from 255 to 0/1)
- Convert class vectors to binary class matices (via one-hot encoding)
- Create validation data set (of 10% of training data)

In [ ]:
# Preprocess the data
def preprocess_data(X_train, y_train, X_test, y_test):
    # Normalize pixel values to be between 0 and 1
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0

    # Convert class vectors to binary class matrices
    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)

    # Create a validation set (10% of training data)
    val_size = int(0.1 * X_train.shape[0])
    X_val = X_train[-val_size:]
    y_val = y_train[-val_size:]
    X_train = X_train[:-val_size]
    y_train = y_train[:-val_size]

    return X_train, y_train, X_val, y_val, X_test, y_test

# Apply preprocessing
X_train, y_train, X_val, y_val, X_test, y_test = preprocess_data(X_train, y_train, X_test, y_test)

# Print updated shapes
print("Training data shape after preprocessing:", X_train.shape)
print("Validation data shape:", X_val.shape)
print("Test data shape:", X_test.shape)

### Let's do a quick visualization of sample images (to also ensure we still have the correct shape)

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i])
    plt.xlabel(label_names[np.argmax(y_train[i])])
plt.tight_layout()
plt.show()


### Let's define our CNN model (architect)
-
-


In [ ]:
# Helper function before your create_cnn_model function
def residual_block(x, filters, use_projection=False):
    """Create a residual block with skip connection"""
    shortcut = x
    
    # First convolution
    x = layers.Conv2D(filters, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # Second convolution (no activation yet)
    x = layers.Conv2D(filters, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    # Adjust shortcut dimensions if needed
    if use_projection or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, (1, 1), padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)
    
    # Add skip connection and apply final activation
    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    
    return x


# Modified create_cnn_model function with residual blocks
def create_cnn_model(input_shape=(32, 32, 3), num_classes=10):
    # Use Functional API instead of Sequential for residual connections
    inputs = layers.Input(shape=input_shape)
    
    # Initial convolution
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    
    # First residual block (32 filters)
    x = residual_block(x, filters=32, use_projection=True)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)

    # Second residual block (64 filters)
    x = residual_block(x, filters=64, use_projection=True)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)

    # Third residual block (128 filters)
    x = residual_block(x, filters=128, use_projection=True)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.5)(x)

    # Global Average Pooling
    x = layers.GlobalAveragePooling2D()(x)

    # Classifier (same as your simplified version)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Create and return the model
    model = models.Model(inputs=inputs, outputs=outputs)
    
    return model

# Create and compile the model
model = create_cnn_model()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Display model summary
model.summary()

In [ ]:
def apply_data_augmentation():
  """Apply data augmentation using tf.image (Keras 3.x compatible)"""

  @tf.function
  def augment_image(image, label):
    # Only horizontal flip (most conservative augmentation)
    image = tf.image.random_flip_left_right(image)

    # Very mild brightness adjustment
    image = tf.image.random_brightness(image, max_delta=0.05)

    # Ensure values stay in [0,1] range
    image = tf.clip_by_value(image, 0.0, 1.0)

    return image, label

  # Create tf.data dataset
  train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
  train_dataset = train_dataset.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
  train_dataset = train_dataset.batch(64).prefetch(tf.data.AUTOTUNE)

  return train_dataset


def apply_mild_data_augmentation():
    """Apply very conservative data augmentation"""
    datagen = ImageDataGenerator(
        horizontal_flip=True,          # Only horizontal flip
        width_shift_range=0.05,        # Very small shifts (5%)
        height_shift_range=0.05,       # Very small shifts (5%)
        rotation_range=5               # Very small rotation (5 degrees)
    )

    # Fit the augmentation parameters to training data
    datagen.fit(X_train)
    return datagen

# Create the mild augmentation generator



In [ ]:
# Define callbacks for early stopping and model checkpoint
from keras.callbacks import EarlyStopping #, ModelCheckpoint
k.backend.clear_session()


callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True) #,
    # ModelCheckpoint('best_cnn_model.keras', monitor='val_accuracy',
    #                save_best_only=True, mode='max')
]


train_dataset = apply_data_augmentation()
#datagen = apply_mild_data_augmentation()

# Train the model
start_time = time.time()
history = model.fit(
    train_dataset,
    #datagen.flow(X_train, y_train, batch_size=64),
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=callbacks
)
end_time = time.time()
print(f"Training time: {end_time - start_time:.2f} seconds")

### It stopped at 24 epochs, it must have not improve for 10 consecutive epochs

In [ ]:
# Plot training history
def plot_training_history(history):
    # Plot accuracy
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_training_history(history)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")
print(f"Test loss: {test_loss:.4f}")

### The CNN model shows:

- Good Learning: Training accuracy reached ~94%, validation ~84-85%
- Moderate Overfitting: ~9-10% gap between training and validation metrics
- Early Plateau: Most learning occurred in first 10 epochs
- Stable Validation: Validation metrics stabilized without deteriorating

The curves show a well-functioning model that's starting to memorize training data rather than learning generalizable features.

In [ ]:
# Predict classes for test data
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Classification report
print(classification_report(y_true_classes, y_pred_classes, target_names=label_names))

### Class Performance

- Overall Accuracy: 84.2% on test set
- Best Classes: Vehicles (ships 92.1%, automobiles 91.8%)
- Struggling Classes: Cats (62.1% correct), birds (75.9%)
- Man-made vs Natural: Vehicles (90% F1) outperform animals (80% F1)

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_names, yticklabels=label_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.show()

### Confusion Matrix patterns:
- Cat-Dog Confusion: 159 cats misclassified as dogs (biggest issue)
- Vehicle Confusion: Some automobile-truck confusion (expected)
- Animal Misclassifications: Widespread confusion among mammals

Pattern: Classes with consistent appearances perform best

In [ ]:
# Visualize some predictions
def visualize_predictions(X, y_true, y_pred, class_names, num_images=25):
    plt.figure(figsize=(12, 12))
    for i in range(num_images):
        plt.subplot(5, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(X[i])

        true_label = class_names[y_true[i]]
        pred_label = class_names[y_pred[i]]
        color = 'green' if true_label == pred_label else 'red'

        plt.xlabel(f"T: {true_label}\nP: {pred_label}", color=color)
    plt.tight_layout()
    plt.show()

# Get a subset of test data
sample_indices = np.random.choice(len(X_test), 25, replace=False)
visualize_predictions(
    X_test[sample_indices],
    y_true_classes[sample_indices],
    y_pred_classes[sample_indices],
    label_names
)

# Save the trained model
model.save('cifar10_cnn_model.keras')
print("Model saved as 'cifar10_cnn_model.keras'")